# Set-TSP - Because There Is More Than One Place to Get Bread

## This notebook was created to serve a blog post by the same name.

In [1]:
# written in python 3.7.3
import numpy as np
from math import cos, asin, sqrt, degrees, atan2
from geojson import LineString, Point, Feature, FeatureCollection
import geojsonio
import json
import random
random_seed = 42
random.seed(random_seed)

# Part I - The Geographic Building Blocks

## For more information on Haversine formula, go to [this](https://en.wikipedia.org/wiki/Haversine_formula) wiki page, 

## and for the specific formulation used here go to [this](https://stackoverflow.com/questions/27928/calculate-distance-between-two-latitude-longitude-points-haversine-formula/21623206#21623206) Stackoverflow answer

In [60]:
p = 0.017453292519943295  # Pi / 180
r = 12742000  # Earth's radius is ~6371km => r = 2 * Earth's radius

class GeoPoint:
    def __init__(self, lng: float, lat: float):
        # Why 5 digits? According to https://en.wikipedia.org/wiki/Decimal_degrees it's 1m. accuracy.
        self.lng = round(lng, 5)
        self.lat = round(lat, 5)
        
    def __repr__(self):
        # Copy-pastable format for most map applications
        return f"[{self.lng}, {self.lat}]"
    
    def get_dist_from(self, other) -> int:
        # Return non-euclidean distance in meters, using Haversine formula
        # For more information on this formulation go to 
        a = (0.5 
             - cos((other.lat - self.lat) * p)/2 
             + (cos(self.lat * p) 
                * cos(other.lat * p) 
                * (1 - cos((other.lng - self.lng) * p)) / 2))
        d = int(r * asin(sqrt(a))) 
        return d
    
    def get_angle_from(self, other) -> float:
        x = other.lng - self.lng
        y = other.lat - self.lat
        a = degrees(atan2(y, x))
        return a


def get_geo_point_of_center(geo_points: [GeoPoint]) -> GeoPoint:
    lng_list, lat_list = list(zip(*[[g.lng, g.lat] for g in geo_points]))
    lng_center, lat_center = np.mean(np.array([lng_list, lat_list]), axis=1)
    return GeoPoint(lng_center, lat_center)


In [6]:
beer_2 = GeoPoint(34.899868, 32.177147)
veg_1 = GeoPoint(34.899793, 32.178663)
bread_2 = GeoPoint(34.907224, 32.175415)
bread_2.get_dist_from(GeoPoint(34.847480, 32.187983))


5793

In [105]:
distances_array = np.array([[0, 600, 550, 600, 650, 400, 150], 
                            [0, 0, 170, 900, 1100, 850, 550], 
                            [0, 0, 0, 750, 1000, 850, 400], 
                            [0, 0, 0, 0, 900, 750, 600], 
                            [0, 0, 0, 0, 0, 260, 700], 
                            [0, 0, 0, 0, 0, 0, 500], 
                            [0] * 7])
distances_array += distances_array.transpose()
distances_array

array([[   0,  600,  550,  600,  650,  400,  150],
       [ 600,    0,  170,  900, 1100,  850,  550],
       [ 550,  170,    0,  750, 1000,  850,  400],
       [ 600,  900,  750,    0,  900,  750,  600],
       [ 650, 1100, 1000,  900,    0,  260,  700],
       [ 400,  850,  850,  750,  260,    0,  500],
       [ 150,  550,  400,  600,  700,  500,    0]])

In [56]:
def euclidean_dist(geo_point_1: GeoPoint, geo_point_2: GeoPoint):
    d = np.linalg.norm(np.array([geo_point_1.lat, geo_point_1.lng])
                       - np.array([geo_point_2.lat, geo_point_2.lng]))
    return d

In [61]:
n_points = 10
geo_points = [GeoPoint(34.8 + (0.1 * random.random()), 32.1 + (0.1 * random.random()))
              for _ in range(n_points)]
distances_array = np.array([[g_to.get_dist_from(g_from)
                             for g_to in geo_points]
                            for g_from in geo_points])

In [62]:
def DP_TSP(n, distances_array):
    all_points_set = set(range(n))

    memo = {tuple([i]): tuple([i, 0]) for i in range(n)}
    queue = [tuple([i]) for i in range(n)]

    while queue:
        prev_visited = queue.pop(0)
        if set(prev_visited) == all_points_set:
            continue

        prev_last_point, prev_dist = memo[prev_visited]

        to_visit = all_points_set.difference(set(prev_visited))
        for new_last_point in to_visit:
            new_visited = tuple(sorted(list(prev_visited) + [new_last_point]))
            new_dist = prev_dist + distances_array[prev_last_point][new_last_point]

            if new_visited not in memo:
                memo[new_visited] = tuple([new_last_point, new_dist])
                queue += [new_visited]
            else:
                if memo[new_visited][1] > new_dist:
                    memo[new_visited] = tuple([new_last_point, new_dist])
                    
    return memo[tuple(range(n))][1]
    

In [63]:
%%time

print(DP_TSP(n_points, distances_array))

26047
Wall time: 8.98 ms


In [107]:
n = 7
all_points_set = set(range(n))

memo = {tuple([i]): tuple([i, 0]) for i in range(n)}
queue = [tuple([i]) for i in range(n)]

while queue:
    prev_visited = queue.pop(0)
    if set(prev_visited) == all_points_set:
        continue
        
    prev_last_point, prev_dist = memo[prev_visited]
    
    to_visit = all_points_set.difference(set(prev_visited))
    for new_last_point in to_visit:
        new_visited = tuple(sorted(list(prev_visited) + [new_last_point]))
        new_dist = prev_dist + distances_array[prev_last_point][new_last_point]
        
        if new_visited not in memo:
            memo[new_visited] = tuple([new_last_point, new_dist])
            print(new_visited, new_last_point, new_dist)
            queue += [new_visited]
        else:
            if memo[new_visited][1] > new_dist:
                memo[new_visited] = tuple([new_last_point, new_dist])
                print(new_visited, new_last_point, new_dist)
    

        





(0, 1) 1 600
(0, 2) 2 550
(0, 3) 3 600
(0, 4) 4 650
(0, 5) 5 400
(0, 6) 6 150
(1, 2) 2 170
(1, 3) 3 900
(1, 4) 4 1100
(1, 5) 5 850
(1, 6) 6 550
(2, 3) 3 750
(2, 4) 4 1000
(2, 5) 5 850
(2, 6) 6 400
(3, 4) 4 900
(3, 5) 5 750
(3, 6) 6 600
(4, 5) 5 260
(4, 6) 6 700
(5, 6) 6 500
(0, 1, 2) 2 770
(0, 1, 3) 3 1500
(0, 1, 4) 4 1700
(0, 1, 5) 5 1450
(0, 1, 6) 6 1150
(0, 1, 2) 1 720
(0, 2, 3) 3 1300
(0, 2, 4) 4 1550
(0, 2, 5) 5 1400
(0, 2, 6) 6 950
(0, 3, 4) 4 1500
(0, 3, 5) 5 1350
(0, 3, 6) 6 1200
(0, 4, 5) 5 910
(0, 4, 6) 6 1350
(0, 1, 5) 1 1250
(0, 2, 5) 2 1250
(0, 3, 5) 3 1150
(0, 4, 5) 4 660
(0, 5, 6) 6 900
(0, 1, 6) 1 700
(0, 2, 6) 2 550
(0, 3, 6) 3 750
(0, 4, 6) 4 850
(0, 5, 6) 5 650
(1, 2, 3) 3 920
(1, 2, 4) 4 1170
(1, 2, 5) 5 1020
(1, 2, 6) 6 570
(1, 3, 4) 4 1800
(1, 3, 5) 5 1650
(1, 3, 6) 6 1500
(1, 4, 5) 5 1360
(1, 4, 6) 6 1800
(1, 3, 5) 3 1600
(1, 4, 5) 4 1110
(1, 5, 6) 6 1350
(1, 3, 6) 3 1150
(1, 4, 6) 4 1250
(1, 5, 6) 5 1050
(2, 3, 4) 4 1650
(2, 3, 5) 5 1500
(2, 3, 6) 6 1350
(2, 4, 

In [108]:
route_reversed = []
visited = list(range(n))
while len(visited) > 0:
    last_p = memo[tuple(visited)][0]
    print(visited, last_p)
    route_reversed += [last_p] 
    visited.remove(last_p) 
    
route_reversed
    

[0, 1, 2, 3, 4, 5, 6] 3
[0, 1, 2, 4, 5, 6] 0
[1, 2, 4, 5, 6] 4
[1, 2, 5, 6] 5
[1, 2, 6] 6
[1, 2] 2
[1] 1


[3, 0, 4, 5, 6, 2, 1]

In [109]:
get_route_len(distances_array, [1, 2, 6, 0, 5, 4, 3])

2280

In [103]:
rice = GeoPoint(34.904145, 32.178397)
veg = GeoPoint(34.899660, 32.178243)
pet= GeoPoint(34.899918, 32.177080)
garden = GeoPoint(34.904370, 32.173966)
pharm = GeoPoint(34.909027, 32.177480)
pasta = GeoPoint(34.906774, 32.178279)
pita = GeoPoint(34.903383, 32.177381)

geo_points = [rice, veg, pet, garden, pharm, pasta, pita]
visualize([1, 2, 6, 0, 5, 4, 3], geo_points)

In [74]:
a=tuple([1,2,3,5])

In [75]:
a = list(a)
a.remove(3)
tuple(sorted(a))

(1, 2, 5)

In [55]:
memo[tuple(range(n))]

(3, 2580)

In [51]:
memo

{(0,): (0, 0),
 (1,): (1, 0),
 (2,): (2, 0),
 (3,): (3, 0),
 (4,): (4, 0),
 (5,): (5, 0),
 (6,): (6, 0),
 (0, 6): (0, 150),
 (1, 6): (1, 550),
 (2, 6): (2, 400),
 (3, 6): (3, 600),
 (4, 6): (4, 700),
 (5, 6): (5, 500),
 (0, 5, 6): (5, 550),
 (1, 5, 6): (1, 1350),
 (2, 5, 6): (5, 1250),
 (3, 5, 6): (3, 1250),
 (4, 5, 6): (4, 760),
 (0, 4, 5, 6): (6, 1060),
 (1, 4, 5, 6): (1, 1860),
 (2, 4, 5, 6): (6, 1660),
 (3, 4, 5, 6): (3, 1660),
 (0, 3, 4, 5, 6): (6, 1910),
 (1, 3, 4, 5, 6): (1, 2560),
 (2, 3, 4, 5, 6): (6, 2310),
 (0, 2, 3, 4, 5, 6): (6, 2610),
 (1, 2, 3, 4, 5, 6): (1, 2580),
 (0, 1, 2, 3, 4, 5, 6): (6, 2830),
 (0, 1, 3, 4, 5, 6): (6, 2810),
 (0, 2, 4, 5, 6): (2, 1960),
 (1, 2, 4, 5, 6): (1, 1930),
 (0, 1, 2, 4, 5, 6): (6, 2180),
 (0, 1, 4, 5, 6): (1, 2010),
 (0, 3, 5, 6): (3, 1500),
 (1, 3, 5, 6): (1, 2150),
 (2, 3, 5, 6): (6, 1900),
 (0, 2, 3, 5, 6): (6, 2200),
 (1, 2, 3, 5, 6): (1, 2170),
 (0, 1, 2, 3, 5, 6): (6, 2420),
 (0, 1, 3, 5, 6): (6, 2400),
 (0, 2, 5, 6): (5, 1350),
 (1,

In [104]:
from itertools import permutations
best_dist = float('inf')
best_routes = []
for p in permutations(range(7)):
    d = get_route_len(distances_array, p)
    if d < best_dist:
        best_dist = d
        best_routes = [list(p)]
    elif d == best_dist:
        best_routes += [list(p)]

print(best_dist, best_routes)

2280 [[1, 2, 6, 0, 5, 4, 3], [3, 1, 2, 6, 0, 5, 4], [3, 2, 1, 6, 0, 5, 4], [3, 4, 5, 0, 6, 2, 1], [4, 5, 0, 6, 1, 2, 3], [4, 5, 0, 6, 2, 1, 3]]


In [36]:
tuple(range(5))

(0, 1, 2, 3, 4)

In [22]:
all_points

{0, 1, 2, 3, 4}

In [24]:
queue

[({0}, 0, 0), ({1}, 1, 0), ({2}, 2, 0), ({3}, 3, 0), ({4}, 4, 0)]

In [25]:
prev_visited_set, prev_last_point, prev_dist = queue.pop()
print(prev_visited_set, prev_last_point, prev_dist)

{4} 4 0


In [27]:
to_visit_list = list(all_points_set.difference(prev_visited_set))
to_visit_list

[0, 1, 2, 3]

In [35]:
visited = set()
visited.add(tuple(sorted(to_visit_list)))
visited

{(0, 1, 2, 3)}

In [19]:
a=set([1,2,3])

In [20]:
a.difference(set([1]))

{2, 3}

In [7]:
def solve_tsp_dynamic(n_points, all_distances):
    #initial value - just distance from 0 to every other point + keep the track of edges
    A = {(frozenset([0, idx+1]), idx+1): (dist, [0,idx+1]) for idx,dist in enumerate(all_distances[0][1:])}
    for m in range(2, n_points):
        B = {}
        for S in [frozenset(C) | {0} for C in itertools.combinations(range(1, n_points), m)]:
            for j in S - {0}:
                B[(S, j)] = min( [(A[(S-{j},k)][0] + all_distances[k][j], A[(S-{j},k)][1] + [j]) for k in S if k != 0 and k!=j])  #this will use 0th index of tuple for ordering, the same as if key=itemgetter(0) used
        A = B
    res = min([(A[d][0] + all_distances[0][d[1]], A[d][1]) for d in iter(A)])
    return res[1]

In [8]:
distances_array = np.array([[0, 600, 550, 600, 650, 400, 150], 
                            [0, 0, 170, 900, 1100, 850, 550], 
                            [0, 0, 0, 750, 1000, 850, 400], 
                            [0, 0, 0, 0, 900, 750, 600], 
                            [0, 0, 0, 0, 0, 260, 700], 
                            [0, 0, 0, 0, 0, 0, 500], 
                            [0] * 7])
distances_array += distances_array.transpose()
distances_array

array([[   0,  600,  550,  600,  650,  400,  150],
       [ 600,    0,  170,  900, 1100,  850,  550],
       [ 550,  170,    0,  750, 1000,  850,  400],
       [ 600,  900,  750,    0,  900,  750,  600],
       [ 650, 1100, 1000,  900,    0,  260,  700],
       [ 400,  850,  850,  750,  260,    0,  500],
       [ 150,  550,  400,  600,  700,  500,    0]])

In [11]:
import itertools
route_idxs = solve_tsp_dynamic(7, distances_array)

In [102]:
# opens a new tab with the route visualization on geojson.io
def visualize(route_idxs: [int], geo_points: [GeoPoint]):
    lng_lat_list = [tuple([geo_points[i].lng, geo_points[i].lat])
                    for i in route_idxs]
    route = Feature(geometry=LineString(lng_lat_list),
                    properties={"name": "This is our route",
                                "stroke": "#8B0000"})
    places = [Feature(geometry=Point(lng_lat), 
                      properties={"name": f"Place {route_idxs[i]}",
                                  "marker-symbol": int(str(route_idxs[i])[-1]),
                                  "marker-color": "#00008B"})
              for i, lng_lat in enumerate(lng_lat_list)]
    
    feature_collection = FeatureCollection(features=[route] + places)
    geojsonio.display(json.dumps(feature_collection));

In [14]:
rice = GeoPoint(34.904145, 32.178397)
veg = GeoPoint(34.899660, 32.178243)
pet= GeoPoint(34.899918, 32.177080)
garden = GeoPoint(34.904370, 32.173966)
pharm = GeoPoint(34.909027, 32.177480)
pasta = GeoPoint(34.906774, 32.178279)
pita = GeoPoint(34.903383, 32.177381)

geo_points = [rice, veg, pet, garden, pharm, pasta, pita]
visualize(route_idxs, geo_points)

In [15]:
def get_route_len(distances_array: np.array, route_idxs):
    route_len = sum([distances_array[i1][i2]
                     for i1, i2 in zip(route_idxs[:-1], route_idxs[1:])])
    return route_len

In [16]:
get_route_len(distances_array, route_idxs)

3030

In [17]:
route_idxs

[0, 5, 4, 3, 1, 2, 6]

In [18]:
get_route_len(distances_array, [3, 6, 0, 5, 4, 2, 1])

2580